### Install Postgres on Raspberry Pi

Follow the instructions in this article: [Setting up a PostgreSQL Database on a Raspberry Pi](https://pimylifeup.com/raspberry-pi-postgresql/)


In [1]:
%%bash
sudo apt update -y
sudo apt install -y postgresql

Hit:1 https://download.docker.com/linux/raspbian buster InRelease
Hit:2 http://archive.raspberrypi.org/debian buster InRelease
Hit:3 http://raspbian.raspberrypi.org/raspbian buster InRelease
Reading package lists...
Building dependency tree...
Reading state information...
98 packages can be upgraded. Run 'apt list --upgradable' to see them.
Reading package lists...
Building dependency tree...
Reading state information...
postgresql is already the newest version (11+200+deb10u4).
0 upgraded, 0 newly installed, 0 to remove and 98 not upgraded.


In [2]:
%%html
<img src="secure-postgres-8x.gif" width="1200px" />